In [1]:
"""
Authors      : Aditya Jain and Safwan Jamal
Date started : November 16, 2022
About        : Convex Optimization project; training script
"""

import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn as nn

from cifardataset import CIFARDataset
from custom_cnn_one import CustomCNN
from custom_cnn_two import CustomCNNTwo

/home/mila/a/aditya.jain/.conda/envs/milamoth_ai/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Load Model

In [2]:
num_classes = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Available device is {device}')
model = CustomCNNTwo(num_classes).to(device)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)

Available device is cuda


#### Load Data

In [3]:
train_set      = './image_list_cifar/train_full.csv'
num_epochs     = 70
early_stopping = 8

In [4]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
batch_size     = 4
class_list     = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
                  'dog', 'frog', 'horse', 'ship', 'truck']
train_root_dir = './cifar-10-images/train/'
test_root_dir  = './cifar-10-images/test/'
test_set       = './image_list_cifar/test.csv'

train_data       = CIFARDataset(train_root_dir, train_set, class_list, transform)
train_dataloader = DataLoader(train_data,batch_size=batch_size, shuffle=True, num_workers=2)
print(len(train_dataloader))

test_data        = CIFARDataset(test_root_dir, test_set, class_list, transform)
test_dataloader  = DataLoader(test_data,batch_size=batch_size, shuffle=True, num_workers=2)

375


#### Load Optimizer and Loss function

In [5]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
loss_func = nn.CrossEntropyLoss()

#### Training

In [6]:
best_test_accuracy = 0.0
early_stop_count   = 0

for epoch in range(num_epochs): 
    # Model Training
    model.train()
    train_epoch_loss = 0.0
    for image_batch, label_batch in train_dataloader:    
        image_batch, label_batch = image_batch.to(device), label_batch.to(device)
        label_batch = label_batch.squeeze_()
        
        # Compute and apply gradients
        optimizer.zero_grad()
        outputs   = model(image_batch)  
        t_loss    = loss_func(outputs, label_batch)
        t_loss.backward()
        optimizer.step()
        train_epoch_loss += t_loss.item()        
        
    print(f'Training loss for epoch {epoch+1} is {train_epoch_loss/len(train_dataloader)}')
    
    # Model Evaluation
    model.eval()
    total_samples   = 0.0
    total_correct   = 0.0
    for image_batch, label_batch in test_dataloader:    
        image_batch, label_batch = image_batch.to(device), label_batch.to(device)  
        label_batch = label_batch.squeeze_()
        outputs = model(image_batch)
        
        # Calculate batch accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_samples += label_batch.size(0)
        total_correct += (predicted == label_batch).sum().item()
    curr_accuracy = (total_correct/total_samples)*100
    print(f'Test accuracy for epoch {epoch+1} is {curr_accuracy}%')
    
    if curr_accuracy > best_test_accuracy:
        best_test_accuracy = curr_accuracy
        print(f'Best test accuracy improved to {best_test_accuracy}%')
        early_stop_count = 0
    else:
        early_stop_count += 1
        
    if early_stop_count==early_stopping:
        print(f'The best test accuracy achieved is {best_test_accuracy}')
        break

Training loss for epoch 1 is 2.30478236579895
Test accuracy for epoch 1 is 14.75%
Best test accuracy improved to 14.75%
Training loss for epoch 2 is 2.30045321337382
Test accuracy for epoch 2 is 13.750000000000002%
Training loss for epoch 3 is 2.2810683142344157
Test accuracy for epoch 3 is 14.499999999999998%
Training loss for epoch 4 is 2.1907650044759115
Test accuracy for epoch 4 is 21.0%
Best test accuracy improved to 21.0%
Training loss for epoch 5 is 2.0603837677637737
Test accuracy for epoch 5 is 27.750000000000004%
Best test accuracy improved to 27.750000000000004%
Training loss for epoch 6 is 1.9833827250798544
Test accuracy for epoch 6 is 27.750000000000004%
Training loss for epoch 7 is 1.8973989140192669
Test accuracy for epoch 7 is 28.499999999999996%
Best test accuracy improved to 28.499999999999996%
Training loss for epoch 8 is 1.8438198628425597
Test accuracy for epoch 8 is 28.999999999999996%
Best test accuracy improved to 28.999999999999996%
Training loss for epoch 9 i

#### Trying out AlexNet

In [7]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet')


Using cache found in /home/mila/a/aditya.jain/.cache/torch/hub/pytorch_vision_v0.10.0


In [8]:
model

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [9]:
list(model.children())[:3, :-1]

TypeError: list indices must be integers or slices, not tuple

In [ ]:
model.classifier.